# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [89]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt
import random

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [48]:
import numpy as np
def solve_tsp_nearest_neighbour(instance):
    last_on_the_way = np.random.randint(0, len(instance) - 1)
    vert_in_ans = 0
    in_answer = np.zeros(len(instance), dtype=int)
    permutation = np.zeros(len(instance), dtype=int)
    in_answer[last_on_the_way] = 1
    permutation[vert_in_ans] = last_on_the_way
    vert_in_ans += 1
    while (vert_in_ans != len(instance)):
        flag = False
        for i in range(len(instance)):
            if (not in_answer[i]):
                curr_dist = euclidean_distance(instance[last_on_the_way], instance[i])
                if (not flag):
                    min_dist = curr_dist
                    flag = True
                    num_min = i
                elif (curr_dist < min_dist):
                    min_dist = curr_dist
                    num_min = i
        in_answer[num_min] = 1
        permutation[vert_in_ans] = num_min
        vert_in_ans += 1
        last_on_the_way = num_min
    return permutation

In [138]:
import random

def solve_tsp_nearest_insertion(instance):
    unused = set(range(len(instance)))
    start = random.choice(list(unused))
    path = [start]
    flag = False
    for i in range(len(instance)):
        if (start != i):
            dist = euclidean_distance(instance[start], instance[i])
            if (not flag):
                min_dist = dist
                flag = True
                num_min = i
            elif (dist < min_dist):
                min_dist = dist
                num_min = i
    path.append(num_min)
    unused.remove(start)
    unused.remove(num_min)
    while (len(path) != len(instance)):
        after_index = 0
        min_dist = 0
        to_add = 0
        flag = False
        for vert in unused:
            for i in range(1, len(path)):
                dist = euclidean_distance(instance[path[i]], instance[vert]) +\
                    euclidean_distance(instance[path[i - 1]], instance[vert]) -\
                    euclidean_distance(instance[path[i]], instance[path[i - 1]])
                if (not flag):
                    min_dist = dist
                    after_index = i - 1
                    flag = True
                    to_add = vert
                elif (dist < min_dist):
                    min_dist = dist
                    after_index = i - 1
                    to_add = vert
        path.insert(after_index + 1, to_add)
        unused.remove(to_add)
    return path

In [142]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [146]:
run_all()

Solving instance d198.tsp… done in 0.088 seconds with tour length 18478 using NN and in 8.3 seconds with tour length 17625 using NI
Solving instance d493.tsp… done in 0.53 seconds with tour length 42852 using NN and in 1.3e+02 seconds with tour length 40407 using NI
Solving instance d657.tsp… done in 0.94 seconds with tour length 63636 using NN and in 3.1e+02 seconds with tour length 58346 using NI
Solving instance d2103.tsp…

KeyboardInterrupt: 

## Выводы
(Опишите в 1-2 предложениях свои наблюдения по результатам запусков.)

Nearest insertion в моей реализации работает очень-очень долго, мне не удалось дождаться окончания выполнения. Кажется, алгоритм nearest insertion должен приближать реальный ответ значительно лучше, чем nearest neighbour.